In [14]:
using POMDPs
using POMDPModels
using POMDPSimulators
using SARSOP
using QMDP
using BasicPOMCP
using DMUStudent
using QuickPOMDPs

# 1D Discrete Representation Experiment

Frame the camera search problem in a 1D Discrete world to work out the kinks of a Julia POMDPs implementation.

In [22]:
struct GroundWatchState
    cam::Int8
    target::Int8
end

In [25]:
s = GroundWatchState(2,2)
println(s.cam)
println(s.target)

2
2


In [19]:
S = GroundWatchState(2,7)
A = [:left, :stay, :right]
O = [:Int8]

1-element Array{Symbol,1}:
 :Int8

In [39]:
# S = GroundWatchState(2,4)
S = [2,4]
# S = [:one, :two, :three]
A = [:left, :right, :stay]
O = [:yes, :no]
γ = 0.95

function T(s, a, sp)
#     return 0.5
    if a == :stay
        return 1.0
    else 
        return 0.5 #reset
    end
end

function Z(a, sp, o)
#     return 1.0
    println(sp)
    println(' ')
    return 0.25
#     if sp[2] == sp[1]
#         if o == :yes
#             return 0.9
#         else
#             return 0.1
#         end
#     else
#         if o ==:no
#             return 0.8
#         else
#             return 0.2
#         end
#     end
end

function R(s) # need to be based on s and a? or can be based on sp?
    return 10.0
#     if s.cam == s.target
#         return 10.0
#     else
#         return -1.0
#     end
end

m = DiscreteExplicitPOMDP(S,A,O,T,Z,R,γ)

2
 
2
 
4
 
4
 
2
 
2
 
4
 
4
 
2
 
2
 
4
 
4
 


┌ Warning: Transition probabilities sum to 2.0, not 1.
│   s = 2
│   a = stay
└ @ POMDPTesting C:\Users\Mike\.julia\packages\POMDPTesting\yzRbv\src\model.jl:51
┌ Warning: Transition probabilities sum to 2.0, not 1.
│   s = 4
│   a = stay
└ @ POMDPTesting C:\Users\Mike\.julia\packages\POMDPTesting\yzRbv\src\model.jl:51


AssertionError: AssertionError: has_consistent_distributions(m)

In [ ]:
S = [:one, :two, :three, :four, :five, :six]
A = [:left, :stay, :right]
O = [:positive, :negative]
γ = 0.99

function T(s, a, sp)
    if s == :healthy 
        if sp ==:IsC
            return 0.02
        elseif sp ==:healthy
            return 0.98
        else
            return 0.0
        end
    elseif s == :IsC 
        if a == :treat 
            if sp ==:healthy
                return 0.6
            elseif sp == s
                return 0.4
            else
                return 0.0
            end
        else
            if sp ==:InvC
                return 0.1
            elseif sp == s
                return 0.9
            else
                return 0.0
            end
        end
    elseif s ==:InvC
        if a ==:treat
            if sp ==:healthy || sp ==:death
                return 0.2
            elseif sp == s
                return 0.6
            else
                return 0.0
            end
        else
            if sp ==:death
                return 0.6
            elseif sp == s
                return 0.4
            else
                return 0.0
            end
        end
    elseif s ==:death
        if sp == :death
            return 1.0
        else
            return 0.0
        end
    else
        print("Should definitely not get here!")
        return 0.0
    end
end

function Z(a, sp, o)
    if a == :test && sp ==:healthy
        if o == :positive
            return 0.05
        else
            return 0.95
        end
    elseif a == :test && sp ==:IsC
        if o == :positive
            return 0.8
        else
            return 0.2
        end
    elseif a == :test && sp ==:InvC
        if o == :positive
            return 1.0
        else
            return 0.0
        end
    elseif a == :treat && (sp ==:IsC || sp ==:InvC)
        if o == :positive
            return 1.0
        else
            return 0.0
        end 
    else
        if o == :negative
            return 1.0
        else
            return 0.0
        end
    end
end

function R(s, a)
    if s ==:death
        return 0.0
    elseif a ==:wait
        return 1.0
    elseif a ==:test
        return 0.8
    elseif a ==:treat
        return 0.1
    else
        println("error error")
        return -99.0
    end
end

b0 = Deterministic(:healthy)
m = DiscreteExplicitPOMDP(S,A,O,T,Z,R,γ,b0,terminals=Set([:death]));

In [ ]:
using QuickPOMDPs

mountaincar = QuickMDP(
    
    function (s, a)        
        x, v = s
        # shouldn't the target not be a part of the state? 
        
        vp = clamp(v + a*0.001 + cos(3*x)*-0.0025, -0.07, 0.07)
        xp = x + vp
        if xp > 0.5
            r = 100.0
        else
            r = -1.0
        end
        return (sp=(xp, vp), r=r)
    end,
    
    actions = [-1., 0., 1.],
    initialstate = (-0.5, 0.0),
    discount = 0.95,
    isterminal = s -> s[1] > 0.5
)